# Integrantes: María Fernanda Arias y Andrés Jiménez

Parte 1: Entender el sitio web objetivo

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import nltk
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
# Descargar stopwords si no están disponibles
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# URL principal
main_url = "https://www.allrecipes.com/recipes-a-z-6735880"

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.6778.265 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
}

In [ ]:
# Listas para almacenar los datos
data = []

# Hacer la solicitud inicial
response = requests.get(main_url, headers=headers)

In [ ]:
# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Parsear el HTML de la página principal
    soup = BeautifulSoup(response.text, 'html.parser')

    # Buscar los enlaces principales
    main_items = soup.find_all('li', class_='mntl-link-list__item')

    # Usar tqdm para mostrar la barra de progreso
    for main_item in tqdm(main_items, desc="Procesando enlaces principales"):
        main_link = main_item.find('a')
        if main_link:
            main_text = main_link.get_text(strip=True)  # Texto del enlace principal
            main_href = main_link['href']  # URL del enlace principal

            # Hacer una solicitud a la URL del enlace principal
            inner_response = requests.get(main_href, headers=headers)

            # Verificar si la solicitud fue exitosa
            if inner_response.status_code == 200:
                # Parsear el HTML de la página interna
                inner_soup = BeautifulSoup(inner_response.text, 'html.parser')

                # Buscar los enlaces internos (según la clase indicada en tu imagen)
                inner_items = inner_soup.find_all('a', class_='comp mntl-card-list-items mntl-universal-card mntl-document-card mntl-card card card--no-image')

                for inner_item in inner_items:
                    inner_text = inner_item.get_text(strip=True) if inner_item else None
                    inner_href = inner_item['href'] if 'href' in inner_item.attrs else None

                    # Agregar datos al conjunto de resultados
                    data.append({
                        'Texto': main_text,
                        'URL': main_href,
                        'Receta': inner_text,
                        'Receta_URL': inner_href
                    })

else:
    print(f"Error al acceder a la página principal: {response.status_code}")

# Crear un DataFrame con los datos recopilados
df = pd.DataFrame(data)

Procesando enlaces principales: 100%|██████████| 378/378 [03:27<00:00,  1.82it/s]


In [ ]:
df

,Texto,URL,Receta,Receta_URL
0,Air Fryer Recipes,https://www.allrecipes.com/recipes/23070/every...,Air Fryer Corn on The Cob21Ratings,https://www.allrecipes.com/recipe/266396/air-f...
1,Air Fryer Recipes,https://www.allrecipes.com/recipes/23070/every...,Air Fryer Peanut Chicken1Rating,https://www.allrecipes.com/air-fryer-peanut-ch...
2,Air Fryer Recipes,https://www.allrecipes.com/recipes/23070/every...,Air Fryer Crispy Onions,https://www.allrecipes.com/air-fryer-crispy-on...
3,Air Fryer Recipes,https://www.allrecipes.com/recipes/23070/every...,Air Fryer Hot Honey Glazed Carrots1Rating,https://www.allrecipes.com/air-fryer-hot-honey...
4,Air Fryer Recipes,https://www.allrecipes.com/recipes/23070/every...,Air Fryer Sauteed Onions,https://www.allrecipes.com/air-fryer-sauteed-o...
...,...,...,...,...
18949,Zucchini Breads,https://www.allrecipes.com/recipes/348/bread/q...,"Zucchini Bread, Pumpkin Style13Ratings",https://www.allrecipes.com/recipe/152103/zucch...
18950,Zucchini Breads,https://www.allrecipes.com/recipes/348/bread/q...,Gluten-Free Zucchini Bread (or Muffins)11Ratings,https://www.allrecipes.com/recipe/244775/glute...
18951,Zucchini Breads,https://www.allrecipes.com/recipes/348/bread/q...,Cherry-Zucchini Bread2Ratings,https://www.allrecipes.com/recipe/277978/cherr...
18952,Zucchini Breads,https://www.allrecipes.com/recipes/348/bread/q...,Savory Zucchini Muffins38Ratings,https://www.allrecipes.com/recipe/204983/savor...


In [ ]:
# Reduciremos los datos a 100
df = df[:100]

In [ ]:
# Agregar nuevas columnas vacías al DataFrame
df['Nombre'] = ''
df['Ingredientes'] = ''
df['Image'] = ''
df['Preparación'] = ''
df['Descripción'] = ''
df['Información_nutricional'] = ''
df['Detalles'] = ''
df['Rating'] = ''
df['Comentarios'] = ''

/tmp/ipython-input-63-2796870401.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Nombre'] = ''
/tmp/ipython-input-63-2796870401.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ingredientes'] = ''
/tmp/ipython-input-63-2796870401.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [ ]:
!pip install beautifulsoup4 selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 28.0 MB/s eta 0:00:00


In [ ]:
# Iterar por cada URL
from IPython.display import Image, display
for i, url in enumerate(tqdm(df['Receta_URL'], desc="Procesando URLs")):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        ### Nombre (Título de la receta principal)
        h1 = soup.find('h1')
        df.at[i, 'Nombre'] = h1.get_text(strip=True) if h1 else 'No encontrado'

        ### Descripción (og:title)

        title = soup.find("meta", {"property": "og:title"})["content"]
        description = soup.find("meta", attrs={"name": "description"})["content"]
        df.at[i, 'Descripción'] = description

        rating = soup.select_one("#mm-recipes-review-bar__rating_1-0")
        df.at[i, 'Rating'] = rating.get_text(strip=True) if rating else 'No encontrado'


        ### Ingredientes
        ingredientes_list = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
        ingredientes_text = [ingrediente.get_text(strip=True) for ingrediente in ingredientes_list]
        df.at[i, 'Ingredientes'] = ', '.join(ingredientes_text) if ingredientes_text else 'No encontrado'

        ### Preparación / Pasos
        pasos_section = soup.find_all("li", class_="mntl-sc-block-group--LI")
        pasos_text = []
        if pasos_section:
            for j, step in enumerate(pasos_section, 1):
                texto = step.get_text(strip=True)
                if texto:
                    pasos_text.append(f"{j}. {texto}")
        df.at[i, 'Preparación'] = ' '.join(pasos_text) if pasos_text else 'No encontrado'

        ### Información Nutricional
        nutrition_table = soup.find("table", class_="mm-recipes-nutrition-facts-summary__table")
        nutricion = []
        if nutrition_table:
            rows = nutrition_table.find_all("tr")
            for row in rows:
                cols = row.find_all("td")
                if len(cols) == 2:
                    label = cols[0].get_text(strip=True)
                    value = cols[1].get_text(strip=True)
                    nutricion.append(f"{label}: {value}")
        df.at[i, 'Información_nutricional'] = ', '.join(nutricion) if nutricion else 'No encontrado'

        ### Detalles adicionales
        detalles = soup.select(".mm-recipes-details__item")
        detalles_text = []
        for item in detalles:
            label = item.select_one(".mm-recipes-details__label")
            value = item.select_one(".mm-recipes-details__value")
            if label and value:
                detalles_text.append(f"{label.get_text(strip=True)}: {value.get_text(strip=True)}")
        df.at[i, 'Detalles'] = ', '.join(detalles_text) if detalles_text else 'No encontrado'

        ### Rating
        rating = soup.select_one("#mm-recipes-review-bar__rating_1-0")
        df.at[i, 'Rating'] = rating.get_text(strip=True) if rating else 'No encontrado'

        ### Comentarios
        comentarios = []
        for r in soup.select(".photo-dialog__item"):
            author = r.select_one(".photo-dialog__profile--linked")
            date = r.select_one(".ugc-review__date")
            text = r.select_one(".ugc-review__text")
            stars = len(r.select(".ugc-review__rating svg.icon-star"))
            if author and date and text and stars > 0:
                comentarios.append(f"{author.text.strip()} ({date.text.strip()} - {stars}/5): {text.text.strip()}")
            if len(comentarios) == 5:
                break
        df.at[i, 'Comentarios'] = '\n'.join(comentarios) if comentarios else 'No se encontraron comentarios'
        ### Imagen
        image_meta = soup.find("meta", property="og:image")
        image_url = image_meta["content"] if image_meta else None
        df.at[i, 'image_url'] = image_url if image_url else 'No disponible'

        if image_url:
            print(f"\n➡️ Imagen de la receta: {df.at[i, 'Nombre']}")
            display(Image(url=image_url, width=300))  # tamaño reducido
        else:
            print("No se encontró imagen.")

    except Exception as e:
        print(f"❌ Error procesando la URL {url}: {e}")
        df.at[i, 'Nombre'] = 'Error'
        df.at[i, 'Ingredientes'] = 'Error'
        df.at[i, 'Image'] = 'Error'
        df.at[i, 'Preparación'] = 'Error'
        df.at[i, 'Descripción'] = 'Error'
        df.at[i, 'Información_nutricional'] = 'Error'
        df.at[i, 'Detalles'] = 'Error'
        df.at[i, 'Rating'] = 'Error'
        df.at[i, 'Comentarios'] = 'Error'

Procesando URLs:   0%|          | 0/100 [00:00<?, ?it/s]


➡️ Imagen de la receta: Air Fryer Corn on The Cob


Procesando URLs:   1%|          | 1/100 [00:00<00:44,  2.22it/s]


➡️ Imagen de la receta: Air Fryer Peanut Chicken


Procesando URLs:   2%|▏         | 2/100 [00:00<00:43,  2.27it/s]

...

Procesando URLs:  98%|█████████▊| 98/100 [00:47<00:00,  2.22it/s]


➡️ Imagen de la receta: Pineapple Teriyaki Chicken


Procesando URLs:  99%|█████████▉| 99/100 [00:48<00:00,  1.79it/s]


➡️ Imagen de la receta: Frozen Peach Bellini


Procesando URLs: 100%|██████████| 100/100 [00:49<00:00,  2.04it/s]


In [ ]:
print(df[['Nombre', 'image_url']].head(3))


                      Nombre  \
0  Air Fryer Corn on The Cob   
1   Air Fryer Peanut Chicken   
2    Air Fryer Crispy Onions   

                                           image_url  
0  https://www.allrecipes.com/thmb/I8WwP6n5uVghfO...  
1  https://www.allrecipes.com/thmb/ZPso-oD-mSkW2H...  
2  https://www.allrecipes.com/thmb/1RR3GTpLuhuOSW...  


In [ ]:
def clean_text(raw_text):
    """
    Preprocesa texto eliminando caracteres especiales, palabras vacías (stopwords),
    y dejando solo palabras relevantes.

    Args:
        raw_text (str): Texto original a procesar.

    Returns:
        str: Texto limpio y preprocesado.
    """
    # Convertir a minúsculas
    raw_text = raw_text.lower()
    # Sustituir caracteres especiales por espacios
    raw_text = re.sub(r'[^a-zA-Z]', ' ', raw_text)
    # Tokenizar texto
    tokens = word_tokenize(raw_text)
    # Filtrar palabras que no sean relevantes (stopwords y palabras cortas)
    meaningful_words = filter(lambda word: word not in stop_words and len(word) > 2, tokens)
    return ' '.join(meaningful_words)

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Combinar columnas relevantes en un solo campo corpus
df['corpus'] = (
    df['Nombre'].fillna('') + ' ' +
    df['Rating'].fillna('') + ' ' +
    df['Descripción'].fillna('') + ' ' +
    df['Ingredientes'].fillna('') + ' ' +
    df['image_url'].fillna('') + ' ' +
    df['Preparación'].fillna('') + ' ' +
    df['Información_nutricional'].fillna('') + ' ' +
    df['Detalles'].fillna('') + ' ' +
    df['Comentarios'].fillna('')
)

# Aplicar función de limpieza a todo el corpus
df['tokens'] = df['corpus'].apply(clean_text)

/tmp/ipython-input-67-4006254799.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['corpus'] = (
/tmp/ipython-input-67-4006254799.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['corpus'].apply(clean_text)


In [ ]:
# Generación de embeddings utilizando TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
recipe_embeddings = tfidf_vectorizer.fit_transform(df['tokens'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def search_similar_recipes(user_query, embedding_matrix, recipe_df, tfidf_vectorizer):
    """
    Busca recetas similares a una consulta proporcionada por el usuario usando similitud de coseno.

    """

    # Preprocess the user query
    processed_query = clean_text(user_query)
    # tfidf expects a string, not a list of tokens
    joined_query = " ".join(processed_query.split())

    # Vectorize the query
    query_embedding = tfidf_vectorizer.transform([joined_query])

    # Calculate cosine similarities
    similarity_scores = cosine_similarity(query_embedding, embedding_matrix).flatten()

    # Add similarity scores to a copy of the DataFrame
    recipe_df = recipe_df.copy()
    recipe_df['similarity'] = similarity_scores

    # Sort and select the top 10 recipes
    top_recipes = recipe_df.sort_values(by='similarity', ascending=False).head(10)

    # Return all relevant columns for printing
    return top_recipes[['Nombre', 'Ingredientes', 'image_url', 'Preparación', 'Información_nutricional', 'Rating', 'similarity', 'Descripción', 'Detalles', 'Comentarios', 'Receta_URL', 'cluster']]

In [ ]:
def cluster_recipes(embedding_matrix, num_clusters=10):
    """
    Realiza agrupamiento (clustering) de recetas basado en los embeddings generados.

    Args:
        embedding_matrix (sparse matrix): Matriz de embeddings generada.
        num_clusters (int): Número de clusters a formar.

    Returns:
        list: Etiquetas de cluster asignadas a cada receta.
    """
    print("Realizando clustering con K-means...")
    kmeans_model = KMeans(n_clusters=num_clusters, random_state=42)
    cluster_labels = kmeans_model.fit_predict(embedding_matrix)
    return cluster_labels

# Agregar etiquetas de cluster a los datos
df['cluster'] = cluster_recipes(recipe_embeddings, num_clusters=10)

Realizando clustering con K-means...


/tmp/ipython-input-69-3356880454.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cluster'] = cluster_recipes(recipe_embeddings, num_clusters=10)


In [ ]:
def example_query_run(user_query, recipe_df, tfidf_vectorizer):
    """
    Ejecuta una consulta de ejemplo y muestra los resultados completos.

    Args:
        user_query (str): Consulta del usuario.
        recipe_df (DataFrame): DataFrame con las recetas ya procesadas.
        tfidf_vectorizer (TfidfVectorizer): Vectorizador entrenado.
    """
    results = search_similar_recipes(user_query, recipe_embeddings, recipe_df, tfidf_vectorizer)
    print("Recetas similares encontradas:\n")

    for _, row in results.iterrows():
        ingredientes = '\n'.join(f"- {i.strip()}" for i in row['Ingredientes'].split(','))
        preparacion = '\n'.join(f"{idx+1}. {p.strip()}" for idx, p in enumerate(row['Preparación'].split('.')) if p.strip())

        print(f"""
Receta: {row['Nombre']}
URL: {row.get('Receta_URL', 'No disponible')}
Similitud: {row['similarity']:.4f}
Cluster: {row.get('cluster', 'N/A')}

Rating: {row['Rating']}

Descripción:
{row['Descripción']}

Ingredientes:
{ingredientes}

Pasos:
{preparacion}
""")
        if row.get('image_url') and row['image_url'] != 'No disponible':
            print("Imagen:")
            display(Image(url=row['image_url'], width=300))
        else:
            print("Imagen no disponible")

        print(f"""

Información nutricional:
{row['Información_nutricional']}

Detalles:
{row['Detalles']}

Comentarios:
{row['Comentarios']}

{'='*60}
""")

# Ejecutar consulta
user_query_example = "chicken with lemon"
example_query_run(user_query_example, df, tfidf_vectorizer)

Recetas similares encontradas:


Receta: Lemon Garlic Butter Chicken Spiedini
URL: https://www.allrecipes.com/lemon-garlic-butter-chicken-spiedini-recipe-8727930
Similitud: 0.4507
Cluster: 2
            
Rating: 4.7

Descripción:
These lemon garlic butter chicken spiedini are air-fried crispy, golden breaded chicken skewers with a light lemon garlic butter drizzle.

Ingredientes:
- 1/2cupextra-virgin olive oil
- 1/4cupwhite wine
- such as Pinot Grigio
- 3tablespoonslemon juice
- divided
- 1teaspoonlemon zest
- 4clovesgarlic
- finely minced
- divided
- 1/4teaspooncrushed red pepper flakes
- 1/4teaspoondriedoregano
- 2teaspoonskosher salt
- divided
- 1teaspoonfreshly ground blackpepper
- divided
- 2poundsskinless boneless chicken thighs
- cut into 1 1/2 inch pieces
- 2/3cupItalian breadcrumbs
- 2/3cuppanko
- 1/2cupgrated Parmesan cheese
- 1/2teaspoongarlic powder
- 6 (6 to 8 inch)skewers
- olive oilcooking spray
- 4tablespoonsbutter
- 2teaspoonsfinely choppedfresh parsley

Pasos:
1. 1
2.



Información nutricional:
636: Calories, 41g: Fat, 21g: Carbs, 43g: Protein

Detalles:
Prep Time:: 15 mins, Cook Time:: 15 mins, Total Time:: 30 mins, Servings:: 6

Comentarios:
WendyH (01/02/2025 - 2/5): Extremely oily and soggy tasting. Flavor of garlic was good. May be better with chicken breast meat, but way too much oil from the marinade.
Russ (11/24/2024 - 4/5): I made these a few evenings ago exactly per the recipe. They were pretty good. For my taste, they just still need *something...although I'll make them again for sure. Next time perhaps some type of paprika, or minced jalapeno...I liked using the skewers, and I will definitely revisit this one. r-



Receta: Air Fryer Chicken Bites
URL: https://www.allrecipes.com/air-fryer-chicken-bites-recipe-8599352
Similitud: 0.4141
Cluster: 2
            
Rating: 4.0

Descripción:
These air fryer chicken bites, seasoned with lemon, garlic, and Italian herbs, are a perfect addition to an appetizer tray, great to add to a salad, or as a



Información nutricional:
299: Calories, 16g: Fat, 1g: Carbs, 36g: Protein

Detalles:
Prep Time:: 10 mins, Cook Time:: 10 mins, Total Time:: 20 mins, Servings:: 4

Comentarios:
No se encontraron comentarios



Receta: Air Fryer Lemon Garlic Parmesan Chicken
URL: https://www.allrecipes.com/air-fryer-lemon-garlic-parmesan-chicken-recipe-8726749
Similitud: 0.4126
Cluster: 2
            
Rating: 5.0

Descripción:
These simple lemon garlic Parmesan chicken thighs are cooked in the air fryer. Seasoned with lemon zest, garlic, and oregano and coated in Parmesan cheese they come out juicy and tender

Ingredientes:
- 1 1/2poundsskinless boneless chicken thighs
- 3clovesgarlic
- minced
- 1teaspoonlemon zest
- 1teaspoonpaprika
- 1teaspoondried oregano
- 1/2teaspoonsalt
- 1/4teaspooncrushed red pepper
- 1/2cupfreshlygrated parmesan cheese
- 1/4cuppanko bread crumbs
- cooking spray

Pasos:
1. 1
2. Gather all ingredients
3. Dotdash Meredith Food Studios 2
4. Preheat an air fryer to 400 degrees F (2



Información nutricional:
365: Calories, 17g: Fat, 8g: Carbs, 46g: Protein

Detalles:
Prep Time:: 20 mins, Cook Time:: 12 mins, Total Time:: 32 mins, Servings:: 4

Comentarios:
No se encontraron comentarios



Receta: Air Fryer Chicken Tenders
URL: https://www.allrecipes.com/air-fryer-chicken-tenders-recipe-7509380
Similitud: 0.3636
Cluster: 2
            
Rating: No encontrado

Descripción:
Air fryer cooked chicken tenders are quick and easy to make and will surely become a family favorite.

Ingredientes:
- cooking spray
- 1largeegg
- 1/2cuppanko bread crumbs
- 2tablespoonscanola oil
- 8chicken tenders (about 1 lb.)
- choppedfresh parsley
- for garnish (optional)
- 2tablespoonshoney mustard sauce

Pasos:
1. 1
2. Preheat air fryer to 350 degrees F (175 degrees C)
3. Coat basket with cooking spray
4. 2
5. Whisk egg in a small bowl
6. In another bowl, stir together panko and oil until loose and crumbly
7. 3
8. Dip each chicken tender into egg, allowing excess to drip off
9. Dip chicken 



Información nutricional:
509: Calories, 31g: Fat, 33g: Carbs, 25g: Protein

Detalles:
Prep Time:: 10 mins, Cook Time:: 12 mins, Total Time:: 22 mins, Servings:: 4, Yield:: 8 chicken tenders

Comentarios:
No se encontraron comentarios



Receta: Air Fryer Chicken Strips
URL: https://www.allrecipes.com/air-fryer-chicken-tenders-recipe-7501758
Similitud: 0.3425
Cluster: 2
            
Rating: No encontrado

Descripción:
These spicy marinated and breaded air fryer chicken tenders will leave you smiling and wanting more, and they can be made gluten-free with a couple of simple substitutions.

Ingredientes:
- 2poundsskinless
- boneless chicken breasts
- 1cupdillpickle juice
- 1cupmilk
- 1tablespoonliquid smoke
- 3largeeggs
- at room temperature
- 1/2cupranch dressing
- 2cupsall purpose flour
- divided
- 1/2cupbread crumbs
- 4teaspoonspaprika
- 1tablespoonwhite pepper
- 2teaspoonsgarlic salt
- 2teaspoonsconfectioners' sugar
- 1teaspoondriedthyme
- 1teaspoondriedbasil
- 1teaspoondriedoregano

...



Información nutricional:
364: Calories, 20g: Fat, 10g: Carbs, 38g: Protein

Detalles:
Prep Time:: 5 mins, Cook Time:: 15 mins, Total Time:: 20 mins, Servings:: 4

Comentarios:
No se encontraron comentarios



Receta: Easy Air Fryer Chicken Breast
URL: https://www.allrecipes.com/recipe/283850/easy-air-fried-chicken-breast/
Similitud: 0.3176
Cluster: 2
            
Rating: 4.9

Descripción:
This air fryer chicken breast is perfectly seasoned outside and air fried until golden, tender, and incredibly juicy inside for a quick and easy dinner.

Ingredientes:
- 1(8 ounce)chicken breast
- 2teaspoonsolive oil
- 1/4teaspoongarlic powder
- or to taste
- salt and freshly ground black pepper to taste

Pasos:
1. 1
2. Gather all ingredients
3. Preheat an air fryer to 360 degrees F (182 degrees C)
4. Allrecipes / Qi Ai 2
5. Brush both sides of chicken breast with olive oil; season with garlic powder, salt, and pepper on one side
6. Place chicken breast, seasoned-side down, into the air fryer basket



Información nutricional:
456: Calories, 17g: Fat, 1g: Carbs, 70g: Protein

Detalles:
Prep Time:: 5 mins, Cook Time:: 20 mins, Additional Time:: 5 mins, Total Time:: 30 mins, Servings:: 1, Yield:: 1 chicken breast

Comentarios:
Jennifer Cox (11/19/2024 - 5/5): I used mesquite marinated checked breast
Deb C (02/11/2022 - 5/5): This recipe is so quick and easy. I seasoned the chicken breast on both sides and then lightly pounded it so that it would be an even thickness which reduced the baking time by a couple of minutes. The chicken breast was delicious and moist.



Receta: Air Fryer Chicken Parmesan
URL: https://www.allrecipes.com/air-fryer-chicken-parmesan-recipe-8698442
Similitud: 0.2950
Cluster: 2
            
Rating: 4.5

Descripción:
The air fryer is perfect for making chicken Parmesan. No need to fuss with deep-frying and the chicken turns out cruspy and golden brown with lots of cheese and sauce.

Ingredientes:
- 2(8-ounce)boneless skinless chicken breasts
- patted dry
- 3/4te



Información nutricional:
582: Calories, 18g: Fat, 45g: Carbs, 57g: Protein

Detalles:
Prep Time:: 15 mins, Cook Time:: 25 mins, Total Time:: 40 mins, Servings:: 4

Comentarios:
No se encontraron comentarios



Receta: Sheet Pan Za'atar Chicken Thighs and Vegetables
URL: https://www.allrecipes.com/sheet-pan-za-atar-chicken-thighs-and-vegetables-recipe-11753376
Similitud: 0.2817
Cluster: 2
            
Rating: No encontrado

Descripción:
This sheet pan za’atar chicken thighs and vegetables is a great introduction to the flavor of za'atar, a warm spice blend from the Middle East. Chicken thighs roast on a bed of zucchini, yellow summer squash, red bell pepper, and red onion, and if you like, cherry tomatoes.

Ingredientes:
- 2yellow squash
- cut into 1-inch chunks
- 2zucchini
- cut into 1-inch chunks
- 1red bell pepper
- cut into 1-inch pieces
- 1red onion
- cut into eighths
- 5tablespoonsolive oil
- divided
- 3tablespoonsza’atar seasoning
- divided
- salt and freshly ground black peppe



Información nutricional:
508: Calories, 32g: Fat, 16g: Carbs, 45g: Protein

Detalles:
Prep Time:: 15 mins, Cook Time:: 30 mins, Total Time:: 45 mins, Servings:: 4

Comentarios:
No se encontraron comentarios



Receta: Air Fryer Peanut Chicken
URL: https://www.allrecipes.com/air-fryer-peanut-chicken-recipe-11728705
Similitud: 0.2582
Cluster: 2
            
Rating: 5.0

Descripción:
In this air fryer peanut chicken recipe, boneless, skinless chicken thighs are marinated in a spicy-sweet peanut marinade. Rice or noodles work with this dish, or cut it into bite-sized pieces and serve over a salad.

Ingredientes:
- 2tablespoonsThai sweet chili sauce
- 1tablespoonsoy sauce
- 1tablespoonSriracha
- or to taste
- 1lime
- zested and juiced
- 1teaspoongranulated garlic
- 1tablespoonmincedfresh ginger
- 1/3cupcreamy peanut butter
- salt to taste
- 6skinless
- boneless chicken thighs
- cooking spray
- 2cupscooked rice
- or as needed
- chopped cilantro
- sliced scallions
- julienne cucumbers
- and



Información nutricional:
727: Calories, 40g: Fat, 37g: Carbs, 60g: Protein

Detalles:
Prep Time:: 15 mins, Cook Time:: 20 mins, Marinate Time:: 30 mins, Total Time:: 1 hr 5 mins, Servings:: 4

Comentarios:
No se encontraron comentarios


